In [1]:
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]="0, 1, 2, 4"

In [2]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
from dask import dataframe as dd
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import roc_auc_score
import optuna
import gc
xgb.__version__

'1.6.2'

In [3]:
!nvidia-smi

Tue Nov 22 11:02:19 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.142.00   Driver Version: 450.142.00   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-DGXS...  On   | 00000000:07:00.0  On |                    0 |
| N/A   36C    P0    37W / 300W |    356MiB / 32499MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-DGXS...  On   | 00000000:08:00.0 Off |                    0 |
| N/A   

In [4]:
!nvidia-smi -L

GPU 0: Tesla V100-DGXS-32GB (UUID: GPU-d752af09-1f62-bf3b-4f70-78b84e9e41f6)
GPU 1: Tesla V100-DGXS-32GB (UUID: GPU-054a4a35-f98a-3ebc-9100-0f697c246b43)
GPU 2: Tesla V100-DGXS-32GB (UUID: GPU-454525d4-bebd-7fb8-0ba3-3b85e2f99abd)
GPU 3: Tesla V100-DGXS-32GB (UUID: GPU-af0fb74e-f5eb-0833-17ff-494cc6bdcee1)


In [5]:
train_x = dd.read_csv('../input/higgs_small_roc/xgtrain_5.csv')
test_x = dd.read_csv('../input/higgs_small_roc/xgval.csv')

In [6]:
train_y = train_x['target']
train_x = train_x[train_x.columns.difference(['target'])]

test_y = test_x['target']
test_x = test_x[test_x.columns.difference(['target'])]

In [7]:
cluster = LocalCUDACluster(n_workers=4)
client = Client(cluster)

In [8]:
dtrain = xgb.dask.DaskDMatrix(client, train_x, train_y)

In [9]:
dtest = xgb.dask.DaskDMatrix(client, test_x, test_y)


In [10]:
num_round = 1000

In [11]:
def objective(trial):
        
    params = {
        'objective': trial.suggest_categorical('objective',['binary:logistic']),
        'tree_method': trial.suggest_categorical('tree_method',['gpu_hist']),  # 'gpu_hist','hist'
        'lambda': trial.suggest_loguniform('lambda',1e-3,10.0),
        'alpha': trial.suggest_loguniform('alpha',1e-3,10.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.3,1.0),
        'subsample': trial.suggest_uniform('subsample', 0.4, 1.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001,0.1),
        #'n_estimators': trial.suggest_categorical('n_estimators', [1000]),
        'max_depth': trial.suggest_categorical('max_depth', [3,5,7,9,11,13,15,17,20]),
        #'random_state': trial.suggest_categorical('random_state', [24,48,2020]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1,300),
        'eval_metric': trial.suggest_categorical('eval_metric',['logloss']),

    }

    output = xgb.dask.train(client, params, dtrain, num_round)
    
    booster = output['booster']  # booster is the trained model
    booster.set_param({'predictor': 'gpu_predictor'})

    predictions = xgb.dask.predict(client, booster, dtest)
    
    predictions = predictions.compute()

    roc = roc_auc_score(test_y, predictions)
    
    return roc

In [12]:
study = optuna.create_study(direction='maximize')

[I 2022-11-22 11:02:40,096] A new study created in memory with name: no-name-3e39f0d4-ae39-433f-8ff4-d984ee3a4fbb


In [13]:
%%time
study.optimize(objective, n_trials=2)

/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[I 2022-11-22 11:03:04,613] Trial 0 finished with value: 0.7881945449972698 and parameters: {'objective': 'binary:logistic', 'tree_method': 'gpu_hist', 'lambda': 0.06078714253656832, 'alpha': 0.032731970079114756, 'colsample_bytree': 0.549202422235106, 'subsample': 0.4975849611872824, 'learning_rate': 0.0015034354898505351, 'max_depth': 13, 'min_child_weight': 188, 'eval_metric': 'logloss'}. Best is trial 0 with value: 0.7881945449972698.
/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[I 2022-11-22 11:03:41,558] Trial 1 finished with value: 0.8043907516272599 and parameters: {'objective': 'binary:logistic', 'tree_method': 'gpu_hist', 'lambda': 1.2309549736100152, 'alpha': 0.0045

CPU times: user 4.22 s, sys: 1.76 s, total: 5.99 s
Wall time: 56.1 s


In [14]:
%%time
study.optimize(objective, n_trials=100)

/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[I 2022-11-22 11:04:07,880] Trial 2 finished with value: 0.806526469177874 and parameters: {'objective': 'binary:logistic', 'tree_method': 'gpu_hist', 'lambda': 0.04864079708732217, 'alpha': 0.001757914379717806, 'colsample_bytree': 0.4740547871053419, 'subsample': 0.6452009965081721, 'learning_rate': 0.009483743557546227, 'max_depth': 11, 'min_child_weight': 175, 'eval_metric': 'logloss'}. Best is trial 2 with value: 0.806526469177874.
/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[I 2022-11-22 11:04:26,873] Trial 3 finished with value: 0.8030585707271398 and parameters: {'objective': 'binary:logistic', 'tree_method': 'gpu_hist', 'lambda': 1.5879739194889615, 'alpha': 3.738755

CPU times: user 4min 2s, sys: 1min 4s, total: 5min 6s
Wall time: 56min 9s


In [15]:
study.best_trial.params

{'objective': 'binary:logistic',
 'tree_method': 'gpu_hist',
 'lambda': 5.547196173251181,
 'alpha': 9.846733566653675,
 'colsample_bytree': 0.8460596492758485,
 'subsample': 0.624410036544098,
 'learning_rate': 0.016838536382710267,
 'max_depth': 11,
 'min_child_weight': 13,
 'eval_metric': 'logloss'}

In [16]:
real_test_x = dd.read_csv('../input/higgs_small_roc/xgtest.csv')

In [17]:
real_test_y = real_test_x['target']
real_test_x = real_test_x[real_test_x.columns.difference(['target'])]

In [18]:
drealtest = xgb.dask.DaskDMatrix(client, real_test_x, real_test_y)

In [19]:
params = study.best_trial.params

output = xgb.dask.train(client, params, dtrain, num_round)
    
booster = output['booster']  # booster is the trained model
booster.set_param({'predictor': 'gpu_predictor'})

predictions = xgb.dask.predict(client, booster, drealtest)
    
predictions = predictions.compute()

roc_auc_score(real_test_y, predictions)

/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)


0.8077271843791392

In [20]:
%%time 
scores = []

for jj in range(15):
    print(jj)
    params = study.best_trial.params
    params['seed'] = 5*jj**3 + 137

    output = xgb.dask.train(client, params, dtrain, num_round)

    booster = output['booster']  # booster is the trained model
    booster.set_param({'predictor': 'gpu_predictor'})

    predictions = xgb.dask.predict(client, booster, drealtest)

    predictions = predictions.compute()

    scores.append(roc_auc_score(real_test_y, predictions))

0


/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)


1


/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)


2


...

13


/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)


14


/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)


CPU times: user 31.3 s, sys: 10.2 s, total: 41.5 s
Wall time: 7min 32s


In [21]:
scores

[0.8074247562664738,
 0.8074238796632487,
 0.8071942878864131,
 0.8079727272039462,
 0.8074459878053012,
 0.8073759221618079,
 0.8071367921070227,
 0.8073548628128997,
 0.8076396492856597,
 0.8075297555920633,
 0.8064598039591679,
 0.8069119955585436,
 0.8073610877393734,
 0.8072892375821719,
 0.8076572596183071]

In [22]:
import numpy as np
np.mean(scores)

0.8073452003494933